# Load

In [1]:
import numpy as np
import pandas as pd

import datatable as dt
import gc
import re

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go

import warnings
warnings.filterwarnings('ignore')

import shap
from scipy import stats
from tqdm import tqdm
import optuna
from optuna.samplers import TPESampler
import lightgbm as lgb
from lightgbm import LGBMClassifier, LGBMModel

from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import LabelEncoder

from collections import defaultdict
import joblib
import json

# these imports are used to convert the tree to PNG
from cairosvg import svg2png
from PIL import Image
from io import BytesIO

from pandas_profiling import ProfileReport

sns.set(style='whitegrid', font_scale=1.5)

INPUT_DIR = '../input/titanic'

TARGET = 'Survived'
RANDOM_STATE = 997
VERSION = 1

In [2]:
full_train_df = pd.read_csv(f'{INPUT_DIR}/train.csv')
full_test_df = pd.read_csv(f'{INPUT_DIR}/test.csv')

# Train set

In [3]:
full_train_df.info()
full_train_df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [4]:
ProfileReport(full_train_df, progress_bar=False).to_widgets()

# Train LGBM

In [5]:
train_df = full_train_df.copy()
test_df = full_test_df.copy()

In [6]:
CATEGORIAL_FEATURES = [
    'Pclass',
    'Name',
    'Sex',
    'Ticket',
    'Cabin',
    'Embarked'
]
FEATURES = train_df.columns.drop(['PassengerId', TARGET])
FEATURES

Index(['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare',
       'Cabin', 'Embarked'],
      dtype='object')

## Define FE

In [7]:
def feature_engineering(df: pd.DataFrame) -> pd.DataFrame:
    df['Age'].fillna(df['Age'].mean(), inplace=True)
    df['Embarked'] = df['Embarked'].fillna(
        pd.Series(np.random.choice(['S', 'C', 'Q'], p=[0.724, 0.189, 0.087], size=len(df))))
    df['Cabin'].fillna('?', inplace=True)
    
    for col_ in CATEGORIAL_FEATURES:
        df[col_] = df[col_].astype('category')
    
    return df

## Train FE

In [8]:
train_df = feature_engineering(train_df)

train_df.info()
train_df.isnull().sum() / len(train_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   PassengerId  891 non-null    int64   
 1   Survived     891 non-null    int64   
 2   Pclass       891 non-null    category
 3   Name         891 non-null    category
 4   Sex          891 non-null    category
 5   Age          891 non-null    float64 
 6   SibSp        891 non-null    int64   
 7   Parch        891 non-null    int64   
 8   Ticket       891 non-null    category
 9   Fare         891 non-null    float64 
 10  Cabin        891 non-null    category
 11  Embarked     891 non-null    category
dtypes: category(6), float64(2), int64(4)
memory usage: 128.5 KB


PassengerId    0.0
Survived       0.0
Pclass         0.0
Name           0.0
Sex            0.0
Age            0.0
SibSp          0.0
Parch          0.0
Ticket         0.0
Fare           0.0
Cabin          0.0
Embarked       0.0
dtype: float64

## Test FE

In [9]:
test_df = feature_engineering(test_df)

test_df.info()
test_df.isnull().sum() / len(test_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   PassengerId  418 non-null    int64   
 1   Pclass       418 non-null    category
 2   Name         418 non-null    category
 3   Sex          418 non-null    category
 4   Age          418 non-null    float64 
 5   SibSp        418 non-null    int64   
 6   Parch        418 non-null    int64   
 7   Ticket       418 non-null    category
 8   Fare         417 non-null    float64 
 9   Cabin        418 non-null    category
 10  Embarked     418 non-null    category
dtypes: category(6), float64(2), int64(3)
memory usage: 59.2 KB


PassengerId    0.000000
Pclass         0.000000
Name           0.000000
Sex            0.000000
Age            0.000000
SibSp          0.000000
Parch          0.000000
Ticket         0.000000
Fare           0.002392
Cabin          0.000000
Embarked       0.000000
dtype: float64

## Tune

In [10]:
def objective(trial):
    X_train, X_valid, y_train, y_valid = train_test_split(
        train_df[FEATURES],
        train_df[TARGET],
        test_size=len(full_test_df) / len(full_train_df),
        random_state=RANDOM_STATE
    )
    lgb_train = lgb.Dataset(
        X_train, y_train, categorical_feature=CATEGORIAL_FEATURES, free_raw_data=False)
    lgb_valid = lgb.Dataset(
        X_valid, y_valid, categorical_feature=CATEGORIAL_FEATURES, free_raw_data=False)

    param = {
        "boosting_type": "gbdt",
        "objective": "binary",
        "metric": "binary_logloss",
        "verbosity": -1,
        "learning_rate": trial.suggest_uniform("learning_rate", 0.01, 1),
        "num_leaves": trial.suggest_int("num_leaves", 5, 127),
        "max_depth": trial.suggest_int("max_depth", 3, 32),
        'min_data_in_leaf': trial.suggest_int("min_data_in_leaf", 16, 127),
        'random_state': RANDOM_STATE,
    }

    model = lgb.train(param,
                      lgb_train,
                      num_boost_round=200,
                      early_stopping_rounds=10,
                      verbose_eval=10,
                      valid_sets=[lgb_train, lgb_valid])
    y_pred = model.predict(X_valid) >= 0.5
    accuracy = accuracy_score(y_valid, y_pred)

    return accuracy


study = optuna.create_study(
    direction="maximize", sampler=TPESampler(seed=RANDOM_STATE))
study.optimize(objective, n_trials=20)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2020-12-08 18:20:53,374] A new study created in memory with name: no-name-5132e447-77e6-49da-91f1-43d625345c59
[I 2020-12-08 18:20:53,457] Trial 0 finished with value: 0.7918660287081339 and parameters: {'learning_rate': 0.451148188357339, 'num_leaves': 106, 'max_depth': 12, 'min_data_in_leaf': 93}. Best is trial 0 with value: 0.7918660287081339.


Training until validation scores don't improve for 10 rounds
[10]	training's binary_logloss: 0.405539	valid_1's binary_logloss: 0.473334
[20]	training's binary_logloss: 0.381259	valid_1's binary_logloss: 0.464568
[30]	training's binary_logloss: 0.367217	valid_1's binary_logloss: 0.463153
[40]	training's binary_logloss: 0.357683	valid_1's binary_logloss: 0.463152
Early stopping, best iteration is:
[31]	training's binary_logloss: 0.366282	valid_1's binary_logloss: 0.462803
Training until validation scores don't improve for 10 rounds
[10]	training's binary_logloss: 0.466946	valid_1's binary_logloss: 0.501731
[20]	training's binary_logloss: 0.431692	valid_1's binary_logloss: 0.48465
[30]	training's binary_logloss: 0.409889	valid_1's binary_logloss: 0.475057
[40]	training's binary_logloss: 0.395859	valid_1's binary_logloss: 0.471372
[50]	training's binary_logloss: 0.386485	valid_1's binary_logloss: 0.468994
[60]	training's binary_logloss: 0.378952	valid_1's binary_logloss: 0.467235
[70]	tra

[I 2020-12-08 18:20:53,648] Trial 1 finished with value: 0.8086124401913876 and parameters: {'learning_rate': 0.09735105884703202, 'num_leaves': 57, 'max_depth': 9, 'min_data_in_leaf': 79}. Best is trial 1 with value: 0.8086124401913876.


Training until validation scores don't improve for 10 rounds
[10]	training's binary_logloss: 0.3129	valid_1's binary_logloss: 0.440529


[I 2020-12-08 18:20:53,761] Trial 2 finished with value: 0.8133971291866029 and parameters: {'learning_rate': 0.4795064893506527, 'num_leaves': 25, 'max_depth': 16, 'min_data_in_leaf': 36}. Best is trial 2 with value: 0.8133971291866029.


...

Training until validation scores don't improve for 10 rounds
[10]	training's binary_logloss: 0.236024	valid_1's binary_logloss: 0.449451
Early stopping, best iteration is:
[5]	training's binary_logloss: 0.324139	valid_1's binary_logloss: 0.418594
Training until validation scores don't improve for 10 rounds
[10]	training's binary_logloss: 0.313903	valid_1's binary_logloss: 0.440649
[20]	training's binary_logloss: 0.275001	valid_1's binary_logloss: 0.457022
Early stopping, best iteration is:
[11]	training's binary_logloss: 0.308977	valid_1's binary_logloss: 0.440104


[I 2020-12-08 18:20:55,368] Trial 16 finished with value: 0.8157894736842105 and parameters: {'learning_rate': 0.12533859661068678, 'num_leaves': 26, 'max_depth': 21, 'min_data_in_leaf': 68}. Best is trial 14 with value: 0.8349282296650717.


Training until validation scores don't improve for 10 rounds
[10]	training's binary_logloss: 0.446363	valid_1's binary_logloss: 0.487458
[20]	training's binary_logloss: 0.408662	valid_1's binary_logloss: 0.469484
[30]	training's binary_logloss: 0.387784	valid_1's binary_logloss: 0.461618
[40]	training's binary_logloss: 0.371726	valid_1's binary_logloss: 0.455402
[50]	training's binary_logloss: 0.36248	valid_1's binary_logloss: 0.454495
[60]	training's binary_logloss: 0.353431	valid_1's binary_logloss: 0.452632
[70]	training's binary_logloss: 0.345122	valid_1's binary_logloss: 0.449106
[80]	training's binary_logloss: 0.336116	valid_1's binary_logloss: 0.446365
[90]	training's binary_logloss: 0.329722	valid_1's binary_logloss: 0.444173
[100]	training's binary_logloss: 0.32458	valid_1's binary_logloss: 0.442857
[110]	training's binary_logloss: 0.319448	valid_1's binary_logloss: 0.442695
Early stopping, best iteration is:
[104]	training's binary_logloss: 0.322341	valid_1's binary_logloss: 

[I 2020-12-08 18:20:55,481] Trial 17 finished with value: 0.8157894736842105 and parameters: {'learning_rate': 0.3736069622263566, 'num_leaves': 62, 'max_depth': 32, 'min_data_in_leaf': 23}. Best is trial 14 with value: 0.8349282296650717.
[I 2020-12-08 18:20:55,575] Trial 18 finished with value: 0.7679425837320574 and parameters: {'learning_rate': 0.5875014374669518, 'num_leaves': 6, 'max_depth': 23, 'min_data_in_leaf': 127}. Best is trial 14 with value: 0.8349282296650717.
[I 2020-12-08 18:20:55,672] Trial 19 finished with value: 0.8325358851674641 and parameters: {'learning_rate': 0.75218139012201, 'num_leaves': 27, 'max_depth': 19, 'min_data_in_leaf': 29}. Best is trial 14 with value: 0.8349282296650717.


Training until validation scores don't improve for 10 rounds
[10]	training's binary_logloss: 0.422336	valid_1's binary_logloss: 0.488821
[20]	training's binary_logloss: 0.407925	valid_1's binary_logloss: 0.487643
Early stopping, best iteration is:
[19]	training's binary_logloss: 0.40849	valid_1's binary_logloss: 0.485803
Training until validation scores don't improve for 10 rounds
[10]	training's binary_logloss: 0.235477	valid_1's binary_logloss: 0.479031
Early stopping, best iteration is:
[3]	training's binary_logloss: 0.350908	valid_1's binary_logloss: 0.435238
Number of finished trials: 20
Best trial:
  Value: 0.8349282296650717
  Params: 
    learning_rate: 0.3732247365582667
    num_leaves: 32
    max_depth: 20
    min_data_in_leaf: 16


## Start training

In [11]:
% % time
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'learning_rate': 0.2,
    'max_depth': 9,
    'num_leaves': 117,
    'min_data_in_leaf': 127,
    'random_state': RANDOM_STATE,
    'verbose': -1
}

params.update(trial.params)

X_train, X_valid, y_train, y_valid = train_test_split(
    train_df[FEATURES],
    train_df[TARGET],
    test_size=len(full_test_df) / len(full_train_df),
    random_state=RANDOM_STATE
)
lgb_train = lgb.Dataset(
    X_train, y_train, categorical_feature=CATEGORIAL_FEATURES, free_raw_data=False)
lgb_valid = lgb.Dataset(
    X_valid, y_valid, categorical_feature=CATEGORIAL_FEATURES, free_raw_data=False)

model = lgb.train(params,
                  lgb_train,
                  num_boost_round=10000,
                  early_stopping_rounds=20,
                  verbose_eval=10,
                  valid_sets=[lgb_train, lgb_valid])

y_pred = model.predict(X_valid) >= 0.5

accuracy = round(accuracy_score(y_valid, y_pred), 4)
print('\n\n>> LGBM accuracy:', accuracy)

UsageError: Line magic function `%` not found.


## Feature Importance

In [ ]:
lgb.plot_importance(model, figsize=(15, 10), importance_type='split')
plt.show()

In [ ]:
lgb.plot_importance(model, figsize=(15, 10), importance_type='gain')
plt.show()

In [ ]:
# show tree and save as png
def save_tree_diagraph(model):
    tree_digraph = lgb.create_tree_digraph(model, show_info=['split_gain', 'internal_count'])

    tree_png = svg2png(tree_digraph._repr_svg_(), output_width=3840)
    tree_png = Image.open(BytesIO(tree_png))

    tree_png.save('tree_digraph.png')

    display(tree_png)
    
save_tree_diagraph(model)

In [ ]:
shap.initjs()

explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_valid)

shap.summary_plot(shap_values, X_valid, plot_type="bar")

# Submit

In [ ]:
test_df[TARGET] = model.predict(test_df[FEATURES]) >= 0.5
test_df[TARGET] = test_df[TARGET].astype(np.int8)
submission_df = test_df[['PassengerId', TARGET]]
submission_df.to_csv('output.csv', index=False)
submission_df